##### Функция выборки и обработки файлов

In [1]:
import os

def analise(file):
    files = os.listdir(path=file)
    person = []
    table = []
    temp = []
    for i in range(len(files)):
        person.append(str(files[i])[:4])
        with open(file+'/'+files[i],'r') as f:
            for line in f:
                line = str(line).replace(' [ ','').replace(' ]','').replace('\n','')
                line = line.split(' ')
                for j in range(len(line)):
                    line[j] = float(line[j])
                table.append(line)
    return person, table

##### Импорт библиотек

In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
# import mglearn
from IPython.display import display
plt.rc('font', family='Verdana')
import sklearn

##### Обработка тестовой выборки

In [3]:
person, table = analise('test_db')
# print(person1[:10])
person = np.array(person)
table = np.array(table)

print(len(person), len(table))
print(person[:10])
print(table[0][:10])

# print(len(table[0]))
# print(len(table[1]))
# print(len(person))
# print(len(table))
# print(person)

953 953
['0095' '0095' '0095' '0095' '0095' '0095' '0095' '0095' '0095' '0096']
[ 2.1516    0.643147  1.1872   -0.396718 -1.92148   1.68661   2.19098
  0.75515  -2.07515   0.736695]


In [6]:
print(person.shape, table.shape)

(953,) (953, 400)


##### Обработка обучающей выборки

In [8]:
person1, table1 = analise('train_db')
# print(person1[:10])
person1 = np.array(person1)
table1 = np.array(table1)

print(len(person1), len(table1))
print(person1[:10])
print(table1[0][:10])

52284 52284
['0001' '0001' '0001' '0001' '0001' '0001' '0001' '0001' '0001' '0001']
[ 1.76204   1.7534   -0.558433  0.746582 -2.34514   1.54503   0.918075
 -0.998263 -1.50152   1.25771 ]


In [9]:
person1.shape, table1.shape

((52284,), (52284, 400))

##### Случайный лес

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

In [11]:
X_train, X_test, y_train, y_test = train_test_split(table1, person1, random_state=0)

forest = RandomForestClassifier(n_estimators=700, random_state=0, n_jobs=-1) #, max_depth=4)

scaled = StandardScaler()
scaled.fit(X_train)

X_train = scaled.transform(X_train)
X_test = scaled.transform(X_test)
X_test1 = scaled.transform(table)

forest.fit(X_train, y_train)

print("Правильность на обучающем наборе: {:.3f}".format(forest.score(X_train, y_train)))
print("Правильность на тестовом наборе: {:.3f}".format(forest.score(X_test, y_test)))

Правильность на обучающем наборе: 1.000
Правильность на тестовом наборе: 0.910


In [13]:
from sklearn.externals import joblib
joblib.dump(forest, 'RandomForest.pkl') 

['RandomForest.pkl']

In [ ]:
forest = joblib.load('RandomForest.pkl')

In [14]:
predict = forest.predict(table)

prover = {}

err1 = []
err1_count = []
t1 = 1

err2 = []
err2_count = []
t2 = 1

for i in range(len(predict)):
    for j in range(len(predict)):
        if i == j:
            continue
            
        if str(i)+str(j) in prover:
            prover[str(i)+str(j)] += 1
            continue
        else:
            prover[str(i)+str(j)] = 1
            prover[str(j)+str(i)] = 1
#             print(len(prover))
       
        if person[i] == person[j]:
            err1_count.append(1)
#             if predict[i] != predict[j]:
            if abs(int(predict[i])-int(predict[j])) > 120:
                err1.append(t1)
                t1 += 1
        if person[i] != person[j]:
            err2_count.append(1)
            if predict[i] == predict[j]:
                err2.append(t2)
                t2 += 1
print(len(err1)/len(err1_count), len(err2)/len(err2_count))

0.03606462781304097 0.021268101391415276
